# 1. Import libraries

In [29]:
import pandas as pd
import numpy as np

import fastf1
from fastf1.ergast import Ergast

from tqdm import tqdm
import os

import sys
sys.path.append('../')

from src.race_prediction_model.extract import *
from src.race_prediction_model.feature_engineering import add_features_to_results
from src.eda_support import * # Revisar qué funciones necesito de aquí

# Config less verbosity
fastf1.set_log_level('ERROR')

# 2. Data loading

Extraemos los circuitos (esto lo necesitamos antes)

In [30]:
# Poner esto también en A1-EDA-Clustering

try:
    df_races = pd.read_csv('../data/output/races.csv')

except FileNotFoundError:
    print("Data not found: Extracting data...")
    df_races = extract_races_dataframe(2010, end=2024, save=True)

In [31]:
df_races

,season,round,circuitId
0,2010,1,bahrain
1,2010,2,albert_park
2,2010,3,sepang
3,2010,4,shanghai
4,2010,5,catalunya
...,...,...,...
300,2024,20,rodriguez
301,2024,21,interlagos
302,2024,22,vegas
303,2024,23,losail


Tenemos 305 carreras desde 2010 hasta 2024. Extraemos los resultados

In [32]:

df_results = pd.DataFrame()

for season in range(2010, 2024): #df_races['season'].unique().tolist():

    df = pd.read_csv(f'../data/output/results_{str(season)}.csv', index_col=0)

    df_results = pd.concat([df_results, df])

df_results.to_csv('../data/output/results.csv')


In [33]:
"""
season = 2024

results_final_df = extract_results_dataframe(df_races[df_races['season'] == season], save=False)[0]

results_final_df.to_csv(f'../data/output/results_{str(season)}.csv')
"""

"\nseason = 2024\n\nresults_final_df = extract_results_dataframe(df_races[df_races['season'] == season], save=False)[0]\n\nresults_final_df.to_csv(f'../data/output/results_{str(season)}.csv')\n"

In [34]:
df = df_results

# 3. Data description

### Contexto

El conjunto de datos objeto de este análisis consiste en una tabla que registra los resultados de todas las carreras de Fórmula 1 desde la temporada 2010 hasta la actualidad.

A lo largo de la historia de la Fórmula 1, el reglamento ha experimentado constantes modificaciones, impulsadas tanto por los avances tecnológicos como por la búsqueda de una mayor seguridad en pista. Estos cambios han generado diferencias sustanciales en las características de las carreras y en el rendimiento de los monoplazas según la época.

Por ejemplo, en 2005 se prohibieron los cambios de neumáticos durante la carrera, salvo en casos de emergencia o condiciones meteorológicas adversas. En 2007, Michelin abandonó el campeonato, dejando a Bridgestone como único proveedor de neumáticos, mientras que en 2011 este papel fue asumido por Pirelli, que continúa como suministrador oficial en la actualidad. En 2010, se eliminaron los repostajes de combustible, obligando a los equipos a iniciar las carreras con el total necesario para completarlas. Asimismo, en 2011 se introdujo el sistema DRS, diseñado para facilitar los adelantamientos, y en 2014 se implementaron los motores híbridos, marcando el inicio de una nueva era tecnológica. 

Estos ejemplos son solo algunos de los múltiples cambios normativos que han ocurrido, pero ilustran cómo los monoplazas y las condiciones de competencia han evolucionado de manera radical en los últimos 15 años. Esto hace que los resultados obtenidos en épocas más antiguas sean poco representativos de la situación actual.

Para este análisis, hemos seleccionado el año 2010 como punto de partida, dado que representa el inicio de una era significativa en la que se eliminó el repostaje de combustible, un cambio que afectó profundamente la estrategia en carrera. Además, en 2010 se introdujo un nuevo sistema de puntuación, muy similar al actual, que otorgó una mayor relevancia a los resultados consistentes y amplió las diferencias entre los puntos otorgados a los primeros clasificados. Si bien podríamos haber optado por 2014, coincidiendo con el inicio de la era híbrida, consideramos que incluir los datos desde 2010 ofrece un mayor volumen de información sin perder relevancia contextual.

### Descripción de los datos

El conjunto de datos ha sido obtenido utilizando la librería `fasf1`, que combina datos de diversas fuentes, incluyendo la API de Ergast, la API de F1 LiveTiming y su propio backend, para recopilar información detallada y precisa (ver documentación).

Para la construcción de un modelo predictivo, se ha recopilado un conjunto de variables clave que proporcionan información crítica sobre el desempeño de los pilotos y equipos en las carreras. Estas variables son:

- **`DriverId`**: Identificador único del piloto.

- **`TeamId`**: Identificador de la escudería a la que pertenece el piloto.

- **`Position`**: Posición final del piloto en la carrera.

- **`GridPosition`**: Posición de salida en la parrilla.

- **`Time`**: Diferencia de tiempo con la que el piloto cruza la meta respecto al líder.

- **`Status`**: Estado final del piloto en la carrera, indicando si terminó o el motivo de su abandono.

- **`Points`**: Puntos obtenidos por el piloto en la carrera.

- **`season`**: Temporada en la que se celebró el Gran Premio.

- **`round`**: Número de ronda dentro de la temporada correspondiente al Gran Premio.

- **`circuitId`**: Identificador único del circuito donde se disputó la carrera.

Este conjunto de variables proporciona una base robusta para explorar patrones, evaluar el rendimiento y construir predicciones precisas sobre futuras competiciones.

Además, se hará uso de *feature engineering* para crear y añadir características clave que permitan capturar información relevante y mejorar el rendimiento del modelo predictivo.

Veamos qué columnas tenemos

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5957 entries, 8 to 20
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DriverId      5957 non-null   object 
 1   TeamId        5957 non-null   object 
 2   Position      5954 non-null   float64
 3   GridPosition  5954 non-null   float64
 4   Time          5913 non-null   float64
 5   Status        5957 non-null   object 
 6   Points        5957 non-null   float64
 7   season        5957 non-null   int64  
 8   round         5957 non-null   int64  
 9   circuitId     5957 non-null   object 
dtypes: float64(4), int64(2), object(4)
memory usage: 511.9+ KB


Vemos que hay algún valor nulo en `Time` y en `GridPosition`, aunque no son muchos. Veamos qué pasa con ellos.

`GridPosition`

In [36]:
df[df['GridPosition'].isna()]

,DriverId,TeamId,Position,GridPosition,Time,Status,Points,season,round,circuitId
9,mazepin,haas,NaN,NaN,167.527,Illness,0.0,2021,22,yas_marina
47,mick_schumacher,haas,NaN,NaN,191.742,Withdrew,0.0,2022,2,jeddah
18,stroll,aston_martin,NaN,NaN,187.603,Withdrew,0.0,2023,15,marina_bay


En el caso de `GridPosition`, las 3 entradas que tenemos se tratan de casos en los que los pilotos no llegaron a clasificarse para la carrera, como podemos confirmar por su status. Podemos directamente eliminar estas entradas.

In [37]:
df.dropna(subset=['GridPosition'], inplace=True)

`Time`

In [38]:
df[df['Time'].isna()].shape

(44, 10)

In [39]:
df[df['Time'].isna()].sort_values(by='Position').head()

,DriverId,TeamId,Position,GridPosition,Time,Status,Points,season,round,circuitId
9,raikkonen,lotus_f1,6.0,4.0,NaN,Finished,8.0,2012,19,americas
3,ricciardo,mclaren,7.0,6.0,NaN,Finished,6.0,2021,1,bahrain
10,grosjean,lotus_f1,7.0,8.0,NaN,Finished,6.0,2012,19,americas
55,sainz,ferrari,8.0,8.0,NaN,Finished,4.0,2021,1,bahrain
17,vergne,toro_rosso,8.0,16.0,NaN,Finished,4.0,2012,16,yeongam


En el caso de `Time` los valores nulos se encuentran en algunos registros de algunas carreras. No parece que sea demasiado relevante ya que son solamente 44 entradas y la posición más alta es un sexto puesto, lo que no afecta a los podios ni victorias.

De todas formas, esto solamente afectaría a un modelo de regresión que prediga precisamente el tiempo de llegada, ya que en el modelo que vamos a construir eliminaremos esta columna.

### Variables categóricas

In [40]:
df.describe(include='O').T

,count,unique,top,freq
DriverId,5954,77,hamilton,280
TeamId,5954,22,ferrari,562
Status,5954,77,Finished,3018
circuitId,5954,35,silverstone,318


`DriverId`

In [41]:
df['DriverId'].unique()

array(['alonso', 'massa', 'hamilton', 'vettel', 'rosberg',
       'michael_schumacher', 'button', 'webber', 'liuzzi', 'barrichello',
       'kubica', 'sutil', 'alguersuari', 'hulkenberg', 'kovalainen',
       'buemi', 'trulli', 'rosa', 'bruno_senna', 'glock', 'petrov',
       'kobayashi', 'grassi', 'chandhok', 'yamamoto', 'heidfeld', 'klien',
       'resta', 'ambrosio', 'maldonado', 'perez', 'karthikeyan',
       'ricciardo', 'raikkonen', 'vergne', 'pic', 'grosjean', 'gutierrez',
       'bottas', 'jules_bianchi', 'chilton', 'garde', 'kevin_magnussen',
       'kvyat', 'ericsson', 'lotterer', 'stevens', 'nasr', 'sainz',
       'max_verstappen', 'merhi', 'rossi', 'jolyon_palmer', 'wehrlein',
       'haryanto', 'vandoorne', 'ocon', 'giovinazzi', 'stroll', 'gasly',
       'brendon_hartley', 'leclerc', 'sirotkin', 'norris', 'albon',
       'russell', 'latifi', 'aitken', 'pietro_fittipaldi', 'tsunoda',
       'mick_schumacher', 'mazepin', 'zhou', 'de_vries', 'sargeant',
       'piastri', 'law

`TeamId`

In [42]:
df['TeamId'].unique()

array(['ferrari', 'mclaren', 'red_bull', 'mercedes', 'force_india',
       'williams', 'renault', 'toro_rosso', 'lotus_racing', 'sauber',
       'hrt', 'virgin', 'lotus_f1', 'marussia', 'caterham', 'manor',
       'haas', 'alfa', 'racing_point', 'alphatauri', 'aston_martin',
       'alpine'], dtype=object)

Con los equipos tenemos un problema ya que algunos equipos son el mismo pero han cambiado de nombre a lo largo de las temporadas. En particular:

'ferrari'

'mclaren'

'red_bull'

'mercedes'

'williams',

'sauber', 'alfa'

'force_india', 'racing_point', 'aston_martin'

'renault', 'lotus_f1', 'alpine'

'lotus_racing', 'caterham'

'virgin', 'marussia', 'manor'

'toro_rosso', 'alphatauri'

'haas'

'hrt'

In [43]:
def team_id_replacement(df):
    dc = {
        'sauber': 'alfa',
        'alfa': 'alfa',
        'force_india': 'aston_martin',
        'racing_point': 'aston_martin',
        'renault': 'alpine',
        'lotus_f1': 'alpine',
        'lotus_racing': 'caterham',
        'virgin': 'manor',
        'marussia': 'manor',
        'toro_rosso': 'alphatauri'
    }
    df['TeamId'] = df['TeamId'].replace(dc)
    return df

In [44]:
df = team_id_replacement(df)

In [45]:
df['TeamId'].unique()

array(['ferrari', 'mclaren', 'red_bull', 'mercedes', 'aston_martin',
       'williams', 'alpine', 'alphatauri', 'caterham', 'alfa', 'hrt',
       'manor', 'haas'], dtype=object)

Ya hemos corregido los nombres de los equipos, veamos los circuitos

`circuitId`

In [46]:
df['circuitId'].unique()

array(['bahrain', 'albert_park', 'sepang', 'shanghai', 'catalunya',
       'monaco', 'istanbul', 'villeneuve', 'valencia', 'silverstone',
       'hockenheimring', 'hungaroring', 'spa', 'monza', 'marina_bay',
       'suzuka', 'yeongam', 'interlagos', 'yas_marina', 'nurburgring',
       'buddh', 'americas', 'red_bull_ring', 'sochi', 'rodriguez', 'baku',
       'ricard', 'mugello', 'portimao', 'imola', 'zandvoort', 'losail',
       'jeddah', 'miami', 'vegas'], dtype=object)

Parece que todo está en orden

### Variables numéricas

In [47]:
df.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
Position,5954.0,11.16,6.22,1.0,6.00,11.00,16.00,24.00
GridPosition,5954.0,10.94,6.27,0.0,6.00,11.00,16.00,24.00
Time,5910.0,106.42,72.66,0.0,35.24,103.18,178.14,232.79
Points,5954.0,4.79,7.13,0.0,0.00,0.00,8.00,50.00
season,5954.0,2016.40,4.10,2010.0,2013.00,2016.00,2020.00,2023.00
round,5954.0,10.56,5.86,1.0,6.00,10.50,15.00,22.00


Aquí no tenemos mucho que comentar

# 4. Feature engineering

Si la carrera no ha ocurrido, no podemos usar información que dependa de los resultados de esa misma carrera, como el tiempo o los puntos finales. En ese caso, debes basarte exclusivamente en información previa o contextual que sí está disponible antes de la carrera. Aquí tienes cómo abordar el problema:

### Redefinir el conjunto de características (features)
Debemos construir nuestras predicciones en base a datos históricos y contextuales disponibles antes de la carrera. Por este motivo añadiremos las sigueintes características:

- `Winner`: Determine if the driver won the race (Position 1)

- `Podium`: Determine if the driver finished on the podium (Position 1, 2, or 3)

- `MeanPreviousGrid`: Mean of the driver's grid positions over the previous races.

- `MeanPreviousPosition`: Mean of the driver's finishing positions over the previous races.

- `CurrentDriverPoints`: Cumulative driver points before the race.

- `CurrentDriverWins`: Cumulative race wins before the race.

- `CurrentDriverPodiums`: Cumulative podium finishes before the race.

- `CurrentTeamPoints`: Cumulative team points before the race.

The average variables are calculated using a default window of 3 races.

In [48]:
add_features_to_results(df)

In [49]:
df

,DriverId,TeamId,Position,GridPosition,Time,Status,Points,season,round,circuitId,DriverPointsCumulative,TeamPointsCumulative,Winner,Podium,WinsCumulative,PodiumsCumulative
8,alonso,ferrari,1.0,3.0,0.000,Finished,25.0,2010,1,bahrain,25.0,25.0,1,1,1,1
7,massa,ferrari,2.0,2.0,16.099,Finished,18.0,2010,1,bahrain,18.0,43.0,0,1,0,1
2,hamilton,mclaren,3.0,4.0,23.182,Finished,15.0,2010,1,bahrain,15.0,15.0,0,1,0,1
5,vettel,red_bull,4.0,1.0,38.799,Finished,12.0,2010,1,bahrain,12.0,12.0,0,0,0,0
4,rosberg,mercedes,5.0,5.0,40.213,Finished,10.0,2010,1,bahrain,10.0,10.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,sargeant,williams,16.0,20.0,87.791,Finished,0.0,2023,22,yas_marina,1.0,26.0,0,0,0,0
24,zhou,alfa,17.0,19.0,89.422,Finished,0.0,2023,22,yas_marina,6.0,16.0,0,0,0,0
55,sainz,ferrari,18.0,16.0,189.422,Retired,0.0,2023,22,yas_marina,178.0,363.0,0,0,1,3
77,bottas,alfa,19.0,18.0,189.422,+1 Lap,0.0,2023,22,yas_marina,10.0,16.0,0,0,0,0


In [50]:
df.to_csv('../data/output/featured_results.csv')

In [51]:
df = pd.read_csv('../data/output/featured_results.csv', index_col=0)

df

,DriverId,TeamId,Position,GridPosition,Time,Status,Points,season,round,circuitId,DriverPointsCumulative,TeamPointsCumulative,Winner,Podium,WinsCumulative,PodiumsCumulative
8,alonso,ferrari,1.0,3.0,0.000,Finished,25.0,2010,1,bahrain,25.0,25.0,1,1,1,1
7,massa,ferrari,2.0,2.0,16.099,Finished,18.0,2010,1,bahrain,18.0,43.0,0,1,0,1
2,hamilton,mclaren,3.0,4.0,23.182,Finished,15.0,2010,1,bahrain,15.0,15.0,0,1,0,1
5,vettel,red_bull,4.0,1.0,38.799,Finished,12.0,2010,1,bahrain,12.0,12.0,0,0,0,0
4,rosberg,mercedes,5.0,5.0,40.213,Finished,10.0,2010,1,bahrain,10.0,10.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,sargeant,williams,16.0,20.0,87.791,Finished,0.0,2023,22,yas_marina,1.0,26.0,0,0,0,0
24,zhou,alfa,17.0,19.0,89.422,Finished,0.0,2023,22,yas_marina,6.0,16.0,0,0,0,0
55,sainz,ferrari,18.0,16.0,189.422,Retired,0.0,2023,22,yas_marina,178.0,363.0,0,0,1,3
77,bottas,alfa,19.0,18.0,189.422,+1 Lap,0.0,2023,22,yas_marina,10.0,16.0,0,0,0,0


---

In [52]:
df = pd.read_csv('../data/output/featured_results.csv', index_col=0)